In [1]:
import pandas as pd

# Read CSV file
gt = pd.read_csv('/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/random.csv')
gt["labelcol"] = "DefaultValue"

In [3]:
PROMPT_CTA = """
 ----
    Knowing that this table is about {domain}
    ----
    {table}
    ----
    On the provided table i want you to answer this question: 
    What is the label of column {num}
    ----
    ***All the possible relations are in this list {relations}.
    Provide only the desired output (1 word) and nothing more! Dont explain yourself and dont provide extra comments! 
""" 

In [4]:
# Open and read the text file
with open("/home/kpanag/Desktop/cpa3/cta/Round1-SOTAB-CTA-Datasets/cta_labels_round1.txt", "r") as file:
    labels = file.read().splitlines()  # Read lines and remove newline characters

# Print the list
print(labels)


['Number', 'Date', 'Book/name', 'BookFormatType', 'Person/name', 'Language', 'IdentifierAT', 'Review', 'ItemAvailability', 'OfferItemCondition', 'price', 'Event/name', 'priceRange', 'Place/name', 'telephone', 'URL', 'currency', 'category', 'EventAttendanceModeEnumeration', 'EventStatusType', 'email', 'Duration', 'Country', 'openingHours', 'faxNumber', 'LocalBusiness/name', 'streetAddress', 'addressLocality', 'addressRegion', 'postalCode', 'Photograph', 'Movie/name', 'Rating', 'MusicRecording/name', 'MusicAlbum/name', 'MusicArtistAT', 'GenderType', 'Product/name', 'weight', 'DeliveryMethod']


In [ ]:
#These needs to be changed
DATASET_PATH = '/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-SCH-Tables/'  
#GT_PATH = '/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/sotab_cpa_validation_round1.csv'
#RELATIONS_FILE = '/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/cpa_labels_round1.txt'
GT_PATH = '/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/random.csv'
#GT_PATH = '/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/sotab_cpa_test_round1.csv'
def read_datasets():
    gt = read_gt()
    # Get distinct table_name values
    distinct_values = gt['table_name'].unique()
    #print(distinct_values)
    
    # Folder path where the files are stored
    folder_path = DATASET_PATH
    
    df = []

    # Loop over the distinct values and try to read the corresponding files
    for table_name in distinct_values:
        path = folder_path+table_name
        table_df = pd.read_json(path, compression='gzip', lines=True)
        df.append(table_df)
    #len(df)
    return df, distinct_values

def read_gt():
    file_path = GT_PATH
    gt = pd.read_csv(file_path)

    #sort approprietely 
    gt['table_name'] = pd.Categorical(gt['table_name'], categories=gt['table_name'].unique(), ordered=True)
    gt_sorted = gt.sort_values(by=['table_name', 'column_index'])
    gt = gt_sorted
    #print(gt)

    return gt

In [6]:

dataframes, fn = read_datasets()
index = dict(zip(fn, dataframes))
len(index)

16

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
model = "qwen2.5:32b-instruct-q3_K_L"
cta_gt = gt 

for i in range(len(gt)):    
    table_name = gt['table_name'][i]
    domain = table_name.split('_')[0]
    num =  gt['column_index'][i]
    table = index[table_name]
    table = table.to_json(orient="records")
    prompt_template = ChatPromptTemplate.from_template(PROMPT_CTA)
    prompt = prompt_template.format(domain=domain,num=num,table=table,relations=labels)
    #print(prompt)
    llm = OllamaLLM(model=model)
    res = f"""{llm.invoke(prompt)}"""
    #print(res)
    cta_gt.loc[i, 'labelcol'] = res

    res = res.replace(" ", "")

Person/name
BookFormatType
Date
Number
Language
IdentifierAT
Rating
Language
currency
BookFormatType
Person/name
IdentifierAT
Publisher/name
ItemAvailability
price
OfferItemCondition
Date
Date
Event/name
EventAttendanceModeEnumeration
EventAttendanceModeEnumeration
EventStatusType
currency
priceRange
Date
EventStatusType
category
Date
Date
EventStatusType
EventAttendanceModeEnumeration
EventAttendanceModeEnumeration
telephone
telephone
telephone
telephone
URL
Rating
addressRegion
addressLocality
Country
postalCode
URL
URL
Rating
Rating
Rating
Date
Movie/name
Movie/name
Rating
Movie/name
Person/name
Rating
Movie/name
Movie/name
Duration
URL
Person/name
Date
Rating
Rating
Duration
Person/name
MusicRecording/name
Duration
MusicAlbum/name
MusicArtistAT
Country
Person/name
Date
Country
Person/name
URL
GenderType
Person/name
Product/name
Review
OfferItemCondition
Product/name
OfferItemCondition
currency
ItemAvailability
priceRange
priceRange
weight


In [8]:
def llm_csv(gt):
    # Specify the file name
    filename = 'data/llm1.csv'
    # Writing to a CSV file
    #gt['label'] = 'test'
    
    #print(gt)
    #need to change the llm output
    gt.to_csv(filename, index=False)

In [9]:
llm_csv(cta_gt)

In [57]:
gt

,table_name,main_column_index,column_index,label,labelcol
0,Book_1carpetcleaning.co.uk_September2020_CPA.j...,0,7,author,DefaultValue
1,Book_1carpetcleaning.co.uk_September2020_CPA.j...,0,2,bookFormat,DefaultValue
2,Book_1carpetcleaning.co.uk_September2020_CPA.j...,0,3,datePublished,DefaultValue
3,Book_1carpetcleaning.co.uk_September2020_CPA.j...,0,4,numberOfPages,DefaultValue
4,Book_1carpetcleaning.co.uk_September2020_CPA.j...,0,5,inLanguage,DefaultValue
...,...,...,...,...,...
81,Product_earpeace.com_September2020_CPA.json.gz,0,4,priceCurrency,DefaultValue
82,Product_earpeace.com_September2020_CPA.json.gz,0,6,availability,DefaultValue
83,Product_earpeace.com_September2020_CPA.json.gz,0,8,ratingValue,DefaultValue
84,Product_earpeace.com_September2020_CPA.json.gz,0,9,price,DefaultValue


In [1]:
GT_PATH = '/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/sotab_cpa_test_round1.csv'
GT_PATH1 = '/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/sotab_cpa_train_round1.csv'
GT_PATH2 = '/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/sotab_cpa_validation_round1.csv'

In [7]:
import pandas as pd

# Define the file path
GT_PATH = "/home/kpanag/Desktop/cpa3/cpa/Round1-SOTAB-CPA-Datasets/sotab_cpa_test_round1.csv"

# Read the CSV file
df = pd.read_csv(GT_PATH)

# Get the number of distinct values in the first column
num_distinct_table_names = df.iloc[:, 0].nunique()

# Print the count
print("Number of distinct table names:", num_distinct_table_names)


Number of distinct table names: 170


In [14]:
from urllib.parse import urlparse

string = "https://www.google1.com/"
parsed = urlparse(string)
is_url = all([parsed.scheme, parsed.netloc])
print(is_url)


True


In [3]:
s = "1-23"
is_number = s.isdigit()
is_number


False

In [ ]:
url = "https://schema.org/EventStatus"
if "https://schema.org/" in url:
    print("yes")